In [ ]:
# importing necessary modules

import requests
import json
import re
import pandas as pd
import MySQLdb
from bs4 import BeautifulSoup

In [ ]:
# Scraping the multiple websites using BeautifulSoup


base_url='http://books.toscrape.com/catalogue/'
current_page='http://books.toscrape.com/catalogue/page-1.html'
load=requests.get(current_page)

name = []
price = []
quantity = []
rating = []

for i in range(50):
    
    data=BeautifulSoup(load.text,'html.parser')
    next_=data.find_all(class_="next")
    data_1=data.find_all(class_="product_pod")
    for j in data_1:
        book_url=base_url+j.h3.a['href']
        load_book=requests.get(book_url)
        book_data=BeautifulSoup(load_book.text,'html.parser')    
        name.append(book_data.h1.string)
        price1 = book_data.find(class_='price_color').string
        price1 = float(re.search('[\d.]+', price1).group())
        price.append(price1)
        quantity1 = book_data.find(class_='instock availability')
        quantity1 = quantity1.contents[-1].strip()
        quantity1 = int(re.search('\d+', quantity1).group())
        quantity.append(quantity1)
        find_entire_class_name = book_data.find(class_='star-rating')
        rating.append(find_entire_class_name.get('class')[1])

    if next_[0]==None:
        break
    next_page=base_url+next_[0].a["href"]
    load=requests.get(next_page)

In [ ]:
# Creating the dataframe

column_names = ['Name_of_the_Book', 'Price_in_Euros', 'Quantity_in_Stock', 'Rating_Out_of_Five']
df = pd.DataFrame(list(zip(name,price,quantity,rating)),columns=column_names)


In [ ]:
#Storing the dataframe into MYSQL database

host='localhost'
user='root'
password='********'
database='Books_Info'

connection = MySQLdb.connect(host, user, password, database)


cursor = connection.cursor()

# creating column list for insertion
cols = ",".join([str(i) for i in df.columns.tolist()])

# Inserting DataFrame records one by one.
for i,row in df.iterrows():
    sql = "INSERT INTO books_details (" + str(cols) + ") VALUES (" + "%s,"*(len(row)-1) + "%s) ;"
    cursor.execute(sql, tuple(row))
    connection.commit()

# Executing query
sql = "SELECT * FROM books_details ;"
cursor.execute(sql)

# Fetching all the records
result = cursor.fetchall()
for i in result:
    print(i)

connection.close()
